<a href="https://colab.research.google.com/github/mhered/robotica_patito_pycon2025/blob/main/training_smolvla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤗 x 🦾: Training SmolVLA with LeRobot Notebook

Welcome to the **LeRobot SmolVLA training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `SmolVLA` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `SmolVLA` policy for 20,000 steps typically takes **about 5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer!

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [ ]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

## Weights & Biases login
This cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging.

In [ ]:
!wandb login

## Install SmolVLA dependencies

In [ ]:
!cd lerobot && pip install -e ".[smolvla]"

## Start training SmolVLA with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `pepijn223/il_gym0`.

2. `--batch_size=64`: means the model processes 64 training samples in parallel before doing one gradient update. Reduce this number if you have a GPU with low memory.

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

In [ ]:
# use this the first time only
!cd lerobot && lerobot-train \
--dataset.repo_id=mhered/recording-test \
--policy.type=smolvla \
--output_dir=outputs/train/smolvla_so100_pato_test1 \
--job_name=smolvla_so100_pato_test \
--policy.device=cuda \
--wandb.enable=true \
--policy.repo_id=lerobot/smolvla_base \
--steps=20000 \
--save_checkpoint=true \
--save_freq=5000 \
--eval_freq=5000 \
--batch_size=64

In [ ]:
# use this when resuming from local training_state. # steps is absolute number
!cd lerobot && lerobot-train \
 --config_path=outputs/train/smolvla_so100_pato_test1/checkpoints/last/pretrained_model/train_config.json \
 --resume=true \
 --policy.device=cuda \
 --wandb.enable=true \
 --steps=6000 \
 --save_checkpoint=true \
 --save_freq=2000 \
 --eval_freq=2000 \
 --dataset.image_transforms.enable=true

## Login into Hugging Face Hub
Now after training is done login into the Hugging Face hub and upload the last checkpoint

In [ ]:
!git config --global credential.helper store
!huggingface-cli login

In [ ]:
!huggingface-cli upload mhered/my_smolvla \
/content/lerobot/outputs/train/smolvla_so100_pato_test1/checkpoints/last/pretrained_model \
--repo-type model --commit-message "Upload last checkpoint"